<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Численное решение уравнения Шредингера путем разложения по собственным состояниям

## Examples – Quantum Mechanics
<section class="post-meta">
By Jonas Tjemsland, Andreas Krogen og Jon Andreas Støvneng.
</section>
Last edited: January 20th 2019

___

## Вступление

В этом блокноте мы будем решать одномерное уравнение Шредингера,

$$i\hbar\frac{\partial\Psi(x, t)}{\partial t} = -\frac{\hbar^2}{2m}\frac{\partial^2 \Psi( x, t)}{\partial x^2}+V(x)\Psi( x, t) $$

численно для произвольного начального условия $\Psi(x, 0)$. Собственные состояния $\psi_n(x)$ и собственные энергии $E_n$ системы определяются путем решения стационарного уравнения Шредингера

$$-\frac{\hbar^2}{2m}\frac{\partial^2 \psi_n(x)}{\partial x^2}+V(x)\psi_n(x) = E_n\psi_n(x),$$

и нормализации результата. Начальное условие $\Psi(x, 0)$ раскладывается на слагаемые $\psi_n(x)$:

$$\Psi(x,0) = \sum_{i}\alpha_i\psi_i(x).$$


В свою очередь, решение в момент времени $t$, $\Psi(x, t)$, задается

$$\Psi(x, t) = \sum_n\alpha_n\psi_n(x)\exp\left(-i\frac{E_n}{\hbar}t\right).$$

В качестве примера мы будем изучать динамику электрона, заданного гауссовым волновым пакетом. Аналогичный пример изучается в нашем блокноте [Распространение одномерного волнового пакета](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/one_dimensional_wave_propagation.ipynb), но с совершенно другим подходом.

Используемая числовая схема разработана и подробно описана в приложении в конце урока. Читателю рекомендуется прочитать это, прежде чем следовать далее.

Мы начинаем с импорта пакетов, установки общих параметров рисунка и определения физических параметров.

In [ ]:
import matplotlib.pyplot as plt
from scipy.linalg.lapack import ssbevd
import numpy as np
from matplotlib import animation

newparams = {'axes.labelsize': 25, 'axes.linewidth': 1, 'savefig.dpi': 200,
             'lines.linewidth': 3, 'figure.figsize': (20, 10),
             'ytick.labelsize': 25, 'xtick.labelsize': 25,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,
             'figure.titlesize': 25, 
             'legend.fontsize': 25, 'legend.frameon': True, 
             'legend.handlelength': 1.5, 'axes.titlesize': 25,
             'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)

hbar = 1.05E-34  # J⋅s. Reduced Plank's constant
m = 9.11E-31     # kg.  Electron mass

## Потенциал

Как упоминалось во введении, мы будем распространять электрон к потенциальному барьеру в одном измерении. Мы будем рассматривать домен $x\in[0,L]$. Давайте использовать $\Delta x = 1\text Å$, что является типичным диаметром атома. В свою очередь, ширина барьера определяется количеством точек дискретизации, из которых он состоит. Мы хотим, чтобы каждая сторона потенциального барьера была большой, чтобы электрон не находился под влиянием барьера или краев при $t=0$. Мы выбираем $N=10$ точек дискретизации для барьера и в 50 раз больше для каждой из сторон. Барьер имеет высоту $V_0=1,5\cdot 1,6\cdot 10^{-19}J = 1,5\text{eV}$.

Поиграйте с другими параметрами и потенциальными барьерами. Код в этой записной книжке работает даже для произвольных потенциалов!

In [ ]:
V0 = 1.5*1.6E-19 # J. Potential height
dx = 1e-10       # m. Discretization step
N = 10           # #. Number of discretization points in the barrier
N_sides = 100*N  # #. Number of discretization points on each side of the barrier

Ntot = N + 2*N_sides # Total number of discretization points
x = np.linspace(0, dx*Ntot, Ntot) # x-axis
# Potential
V = np.array([0]*N_sides + [V0]*N + [0]*N_sides)

## Волновой пакет

Мы будем представлять начальное состояние электрона в виде гауссова волнового пакета,
$$\Psi(x,0)=C\exp\left(-\frac{(x-x_0)^2}{4\sigma^2}+i\frac{p_0x}{\hbar}\right),$$
где $p_0=\sqrt{2mE_0}$ - импульс волнового пакета, $E_0$ - энергия электрона, $x_0$ - начальное математическое ожидание, а $\sigma$ - некоторый параметр, определяющий ширину волнового пакета. 

Не будет лишним выбрать $E_0\sim V_0$. Как мы увидим, это даст хорошую визуализацию передачи и отражения. Мы начинаем с выбора энергии, немного превышающей потенциальную высоту, $E_0=1,39 V_0$. $x_0$, которая должна находиться в середине левой части области и $\sigma$ (одно стандартное отклонение) до 1/8 левой части. Поиграйте с разными параметрами!

In [ ]:
x0 = 0.5*dx*N_sides
E0 = 1.390*V0
k0 = np.sqrt(2.0*m*E0)/hbar 
sigma = dx*N_sides/8.
A = (2*np.pi*sigma**2)**(-0.25)
Psi_0 = A * np.exp(-(x-x0)**2/(4*sigma**2)) * np.exp(1j*k0*x)
# Check if the wave function is normalized
print("Normalization:", dx*np.sum(np.abs(Psi_0)**2))

Теперь мы визуализируем начальный волновой пакет и потенциал (с подходящим масштабированием).

In [ ]:
plt.plot(x, .75*V*np.max(np.abs(Psi_0)**2)/max(1e-30,np.max(V)), '--')
plt.plot(x, np.abs(Psi_0)**2)
plt.title('Initial probability distribution and potential')
plt.xlabel('$x$ [m]')
plt.ylabel('$|\Psi(x,0)|^2$')
plt.show()

## Решение задачи на собственные значения (уравнение Шредингера)

Теперь, когда все параметры определены, мы можем, наконец, решить уравнение Шредингера. Это делается путем решения задачи на собственные значения. Мы используем солвер для симметричных диагональных матриц (вы, конечно, также можете использовать [numpy.linalg.eigh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eigh.html), но для этого требуется инициализация всей матрицы, в основном состоящей из нулей). Таким образом, нам необходимо инициализировать диагональ, суб - и супердиагональ. Это подробно объясняется в приложениях.

Обратите внимание, что это наиболее сложная с точки зрения вычислений часть задачи.

In [ ]:
diag = hbar**2/(m*dx**2) + V # Diagonal
sup_diag = np.ones(Ntot)*(-hbar**2/(2*m*dx**2)) # Superdiagonal

In [ ]:
E, psi_n, _ = ssbevd([sup_diag, diag]) # Call solver

Давайте визуализируем некоторые собственные состояния и собственные энергии!

In [ ]:
for i in [0, 1, 3]:
    plt.plot(x, psi_n[:,i], label=r"$\psi_{%.0f}(x)$"%(i))
plt.plot(x, .75*V*np.max(psi_n[1])/max(1e-30,np.max(V)), '--', label="Potential")
plt.title("Eigenmodes for the given potential")
plt.xlabel("$x$ [m]")
plt.ylabel("$\psi_n(x)$")
plt.xlim([0,dx*Ntot])
plt.legend()
plt.show()

In [ ]:
plt.plot(E/1.6E-19)
plt.title('Energies')
plt.xlabel('$n$')
plt.ylabel('Energy (eV)')
plt.show()

Вот небольшой вопрос для читателя: почему $\psi_0(x)$ и $\psi_1{x}$ почти равны в правой части домена? Будем ли мы ожидать того же результата для других пар $(\psi_n, \psi_{n+1})$? Подсказка: вырождение.

## Нахождение коэффициентов разложения

Теперь мы рассчитываем коэффициенты разложения, как описано во введении и в приложениях.

In [ ]:
psi_n = psi_n.astype(complex)
c = np.zeros(Ntot, dtype=complex)
for n in range(Ntot):
    c[n] = np.vdot(psi_n[:,n], Psi_0)

## Вычисление $\Psi(x,t)$

Теперь все настроено на вычисление волновой функции в некоторый произвольный момент времени $t$ с учетом начального состояния и потенциала. Для этого мы создаем функцию, выполняющую вычисления, как описано во введении и в приложениях.

In [ ]:
def Psi(t, c, psi_n, E):
    """ Calculate the wave function at some time t given the 
    expansion coefficients c, eigenstates psi_n and
    eigenenergies E.
    
    Parameters:
    -----------
    t :      float. Time
    c :      1d array-like float, len Ntot. Expansion coefficient
    psi_n :  1d array-like float, len Ntot. Eigenstates
    E :      1d array-like float, len Ntot. Eigenenergies
    
    Returns:
    --------
    Numpy-array float, len Ntot. Wave function at time t
    """
    return np.dot(psi_n,c*np.exp(-1j*E*t/hbar))

## Нахождение подходящего временного шага - теорема Эренфеста

Чтобы найти подходящий временной шаг $\Delta t$, мы будем использовать [теорему Эренфеста](https://en.wikipedia.org/wiki/Ehrenfest_theorem). То есть квантово-механические математические ожидания подчиняются классическим уравнениям движения. При нулевом потенциале (математическое ожидание) частица, таким образом, будет иметь скорость
$$v = \frac{p(x)}{m} = \sqrt{\frac{2E_0}{m}}.$$
Таким образом, мы будем использовать $\Delta t \sim \sqrt{m/2E_0}\Delta x$.

Давайте построим график результата для некоторых $t$! 

In [ ]:
dt = 250*dx*(m/(2*E0))**.5
nt = 5
for t in np.arange(0, nt*dt, dt):
    plt.plot(x, np.abs(Psi(t, c, psi_n, E))**2, label=r"$t=%.1e$ s"%(t))
plt.title("Wave function for different $t$")
plt.xlabel("$x$ [m]")
plt.ylabel("$|\Psi(x, t)|^2$")
plt.legend()
plt.show()

## Туннелирование, отражение и передача

Есть много вещей, которым можно научиться из этого простого упражнения. Например, обратите внимание, что мы использовали энергию, которая превышает потенциальный барьер, $E_0>V_0$. В классической механике мы ожидали бы полной передачи, но из приведенного выше графика мы видим, что существует вероятность отражения! С другой стороны, если $E_0<V_0$, мы бы классически ожидали полного отражения, но есть некоторая вероятность передачи (проверьте сами)! Это называется туннелированием. Эти концепции более подробно объясняются в нашей записной книжке по [Распространению одномерных волновых пакетов](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/one_dimensional_wave_propagation.ipynb), где различные вероятности вычисляются явно.

Обратите внимание, что волновая функция имеет высокий пик на барьере. Это опять-таки связано с отражением и передачей. В квантовой механике у нас будет некоторое отражение как при понижении, так и при повышении потенциала (проверьте сами с помощью потенциальной ямы!). Таким образом, пик обусловлен многократным отражением конструктивной интерференции между различными частями волновой функции.

## Упражнения и дальнейшая работа

Исследуйте проблему дальше самостоятельно!

- Каковы преимущества и недостатки использования этого метода (в отличие от более прямого метода, используемого в нашей записной книжке [One-Dimensional Wave Packet Propagation](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/one_dimensional_wave_propagation.ipynb))?
- Найдите численно коэффициент пропускания и отражения для различных ширин барьеров и различных барьеров.
- Реализуйте периодические граничные условия. (Подсказка: Взгляните на матрицу в приложениях и рассмотрите граничные условия по краям. Нам нужно добавить два новых ненулевых матричных элемента. Они расположены в правом верхнем и левом нижнем углу. Обратите внимание, что нам также необходимо использовать разреженную матрицу или общий решатель собственных значений, например, numpy.eigh)
- Объясните, почему у нас есть дисперсия волнового пакета (он распространяется).
- Вычислите (вы можете сделать приближения, если это необходимо), сколько времени требуется электрону, чтобы пройти барьер, отразить правильную границу, снова пройти барьер и вернуться в исходное положение. Проверьте свои вычисления, используя коды Python в этой записной книжке.
- Обобщите метод на два измерения. (Подсказка: Используйте тот же метод конечных разностей, что и в приложении к двумерному уравнению Шредингера. Для простоты используйте $\Delta x = \Delta y = h$. Чтобы записать полученное приближение в виде матрицы, используйте переиндексирование $i,j\to i + (j-1)N$. Внимательно относитесь к границам! Самым простым граничным условием, вероятно, является граничное условие Дирихле.)

## Анимация
Давайте сделаем анимацию, чтобы визуализировать распространяющийся электрон! Также может быть полезно рассчитать вероятности того, что частица окажется в разных частях области. Когда частица прошла через барьер, вероятность того, что частица находится по правую сторону барьера, должна быть приблизительно равна коэффициенту пропускания.

In [ ]:
from matplotlib import animation
from IPython.display import HTML
plt.rcParams.update({'animation.html':'html5', 'savefig.dpi': 50})
            
    
dt=10*dx*(m/(2*E0))**.5
    
def init_anim():
    """ Initialises the animation. """
    global ax, line, textbox
    line, = ax.plot([], [])
    ax.set_xlim([0, dx*Ntot])
    ax.set_ylim([0, 4*np.max(np.abs(Psi_0)**2)])
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('Numerical simulation')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    # A text box that will display the probability for different parts of the domain
    textbox = ax.text(0.05, 0.95, '', transform=ax.transAxes, fontsize=25,
                          verticalalignment='top', bbox=props)
    return line, textbox
    
def animate(i):
    """ Animation function. Being called repeatedly. """
    global ax, line, textbox
    prob = np.abs(Psi(i*dt, c, psi_n, E))**2
    line.set_data(x, prob)
    left_text = "Left side: %.4f\n"%(dx*np.sum(prob[0:N_sides]))
    barrier_text = "Barrier: %.4f\n"%(dx*np.sum(prob[N_sides:N_sides+N]))
    norm_text = "Normalization: %.4f\n"%(dx*np.sum(prob))
    right_text = "Right side: %.4f\n"%(dx*np.sum(prob[-N_sides:]))
    textbox.set_text('Probabilities\n'+norm_text+left_text+right_text+barrier_text)
    return line, textbox

# Run the simulation and visualize the system as an animation.
fig, ax = plt.subplots()
h_anim = animation.FuncAnimation(fig, animate, init_func=init_anim, frames=1000, interval=20, blit=True)
plt.close(h_anim._fig)
HTML(h_anim.to_html5_video())

___
## Приложение: Разработка численной схемы

### Разделение переменных
(Зависящее от времени) уравнение Шредингера гласит

$$i\hbar\frac{\partial\Psi(\vec x, t)}{\partial t} = -\frac{\hbar^2}{2m}\nabla^2\Psi(\vec x, t)+\hat{V}(\vec x, t)\Psi(\vec x, t),$$

для некоторого потенциала $\hat V$. В общем, этот потенциал может зависеть, среди прочего, от времени, скорости и положения. Мы будем рассматривать независимый от времени потенциал, который зависит только от позиции $\vec x$, $V(\vec x)$.

Предположим, что волновая функция $\Psi(\vec x, t)$ может быть записана как произведение одной независимой от времени волновой функции $\psi(\vec x)$ и зависящей от времени функции $\phi(t)$. Это основное предположение в методе разделения переменных для решения дифференциальных уравнений. Затем мы можем переписать уравнение Шредингера следующим образом

$$i\hbar\frac{\phi'(t)}{\phi(t)} = -\frac{\hbar^2}{2m}\frac{\nabla^2\psi(\vec x)}{\psi(\vec x)} + V(\vec x),$$

где мы определили $\phi'$ как производную от $\phi$ по $t$. Обратите внимание, что левая сторона полностью зависит от $t$, в то время как правая сторона зависит только от $\vec x$. Поскольку это уравнение должно выполняться для всех $t$ и $\vec x$, оно должны быть равным некоторой константе, скажем, $E$ (этот выбор скоро станет очевидным; на самом деле это  собственные энергии независимого от времени уравнения Шредингера). Таким образом, мы переписали уравнение Шредингера в

$$ \begin{aligned}
-\frac{\hbar^2}{2m}\nabla^2\psi_n(\vec x) + V(\vec x)&= E_n\psi_n(\vec x),\qquad\qquad&(*) \\
\frac{\phi'(t)}{\phi(t)} &= -\frac{i E_n}{\hbar}, \qquad\qquad&(\dagger)
\end{aligned} $$

Второе уравнение ($\dagger$) легко решается интегрированием. Решение заключается в следующем

$$\phi(t)=\phi_0\exp\left(-\frac{iE_nt}{\hbar}\right),$$

для некоторого начального условия $\phi_0$. Первое уравнение ($*$) мы признаем независимым от времени уравнением Шредингера! Все, что остается - это решить независимое от времени уравнение Шредингера, чтобы найти собственные векторы $\psi_n(\vec x)$ с соответствующими собственными значениями $E_n$.

### Дискретизация - 1D уравнение Шредингера

Рассмотрим теперь одномерный случай, $\vec x\to x$. Пусть система ограничена некоторой областью $x\in[0,L]$. Теперь мы дискретизируем домен в сетку размером $n_x$ с постоянным шагом $\Delta x$. То есть $\left\{x_i\right\}_{i=0,1,...n_x-1}$, где $x_{i+1}-x_i=\Delta x$. 

Обозначим $\psi_i\equiv\psi(x_i)$ и $V_i\equiv V(x_i)$. Используя [метод конечных разностей](https://en.wikipedia.org/wiki/Finite_difference_method), мы можем аппроксимировать независимое от времени уравнение Шедингера $(*)$ следующим образом
$$-\frac{\hbar^2}{2m}\frac{\psi_{i+1}-2\psi_{i}+\psi_{i-1}}{\Delta x^2}+V_i\psi_i= E_n\psi_i.$$

Кроме того, нам нужно тщательно продумать границы. Мы можем, например, использовать граничное условие Дирихле $\psi(0)=\psi(L)=0$, граничное условие Неймана $\psi'(0)=\psi'(L)=0$ или периодические граничные условия, $\psi_{i+n_x}=\psi_{i}$. Мы будем использовать граничное условие Дирихле. В нашем случае это будет моделировать потенциал, где $V(x)=\infty$ для $x\not \in [0,L]$. Другими словами, частица в коробке! Таким образом, мы имеем
$$\psi_0=\psi_{n_x}=0,$$
и нам не нужно рассматривать их как неизвестные.

### Матричное представление
Конечно-разностное приближение в последнем разделе может быть записано в виде матричного уравнения
$$A
\begin{pmatrix}
\psi_1\\ \psi_2 \\ \vdots \\ \psi_{n_x-1}
\end{pmatrix} =
E_n\begin{pmatrix}
\psi_1\\ \psi_2 \\ \vdots \\ \psi_{n_x-1}
\end{pmatrix},
$$
где $A$ - матрица
$$A=
\begin{pmatrix}
\frac{\hbar^2}{m\Delta x^2} + V_i & -\frac{\hbar^2}{2m\Delta x^2} & 0 & 0 & \cdots & 0 & 0 \\
-\frac{\hbar^2}{2m\Delta x^2} & \frac{\hbar^2}{m\Delta x^2} + V_i & -\frac{\hbar^2}{2m\Delta x^2} & 0 & \cdots&0&0\\
0 & -\frac{\hbar^2}{2m\Delta x^2} & \frac{\hbar^2}{m\Delta x^2} + V_i & -\frac{\hbar^2}{2m\Delta x^2}& \cdots&0&0\\
\vdots & \vdots & \ddots & \ddots & \ddots & \vdots & \vdots\\
0 & 0 & 0 & 0 & -\frac{\hbar^2}{2m\Delta x^2} & \frac{\hbar^2}{m\Delta x^2} + V_i & -\frac{\hbar^2}{2m\Delta x^2}\\
0 & 0 & 0 & 0 & 0 & -\frac{\hbar^2}{2m\Delta x^2} & \frac{\hbar^2}{m\Delta x^2} + V_i
\end{pmatrix}.
$$
Поскольку это симметричная трехдиагональная матрица, достаточно использовать два вектора для хранения всей информации в матрице: один длиной $n$ (где $n=n_x-2$), содержащий диагональ, и один длиной $n-1$, содержащий побочную диагональ.

Это может быть легко решено с помощью некоторого решателя собственных значений (например [numpy.linalg.eigh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eigh.html#numpy.linalg.eigh) или какой-нибудь низкоуровневой оберткой от LAPACK [scipy.linalg.lapack](https://docs.scipy.org/doc/scipy-0.18.1/reference/linalg.lapack.html#module-scipy.linalg.lapack)). Мы будем использовать [scipy.linalg.lapack.ssbevd](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.linalg.lapack.ssbevd.html#scipy.linalg.lapack.ssbevd), который является низкоуровневой оболочкой для решателя собственных значений трехдиагональных матриц [LAPACK](http://www.netlib.org/lapack/). Преимущество использования этого метода заключается в том, что достаточно хранить матрицу в виде двух векторов (и, таким образом, снижать использование памяти), и, что более важно, не будем делать лишние проходы по нулевым элементам!

### Реализация

Зависящее от времени уравнение Шредингера берется численно путем решения приведенного выше матричного уравнения. Результатом будет $N$ различных собственных векторов $\psi_n(\vec x)$, соответствующих $N$ собственным значениям $E_n$ (собственные значения могут быть вырожденными). Предположим, что собственные векторы для данного потенциала являются полным набором. Затем мы можем расширить некоторое начальное условие $\Psi(\vec x, 0)$ в терминах собственных векторов $\psi_n(\vec x)$:

$$\Psi(\vec x, 0) = \sum_{n=0}^{N-1}\alpha_n\psi_{n}.$$

Коэффициенты нормировки определяются путем проецирования $\Psi(\vec x, 0)$ на заданный собственный вектор $\psi_n(\vec x)$, путем
$$\alpha_n = \langle \Psi_n(\vec x, 0), \psi_n(\vec x) \rangle\equiv\int_\Omega \Psi(\vec x,0)^*\psi_n(\vec x)\;\text d \tau.$$

Теперь это простая задача - найти решение в произвольное время $t$, учитывая начальное условие $\Psi(\vec x,0)$. Это просто
$$\Psi(\vec x, t) = \sum_{n=0}^{N-1}\alpha_n\psi_n(x)\exp\left(-iE_n t/\hbar\right).$$